### Get Gene Info

In [1]:
import pandas as pd
from Bio import Entrez
import time
import Request_Utilis
Entrez.email = "a.antonakoudis@sartorius.com"

##### Human

In [6]:
gene_name = 'A3GALT2'
search_results = Entrez.esearch(db="gene", term=f"{gene_name}[Gene Name] AND Homo sapiens[Organism]")
record = Entrez.read(search_results)
for gene_name_EntrezID in record['IdList'][0:]:
    gene_info = Entrez.efetch(db="gene", id=gene_name_EntrezID, retmode="xml") 
    gene_record = Entrez.read(gene_info)
    org = gene_record[0]['Entrezgene_source']['BioSource']['BioSource_org']['Org-ref']['Org-ref_taxname']
    if 'Gene-ref_syn' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
        gene_synonyms = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_syn']
    else:
        gene_synonyms = []
    print("Organism: ", org, "|Entrez ID: ", gene_name_EntrezID, "|Gene Name: ", gene_name, "|Gene Synonyms: ", gene_synonyms)
    for assembly_specific_info in gene_record[0]['Entrezgene_locus']:  
        if 'Gene-commentary_heading' in assembly_specific_info:
            for assembly_specific_transcript in assembly_specific_info['Gene-commentary_products']:
                if 'Gene-commentary_products' in assembly_specific_transcript:
                    transcript_sequence_ID = assembly_specific_transcript['Gene-commentary_accession']
                    print(transcript_sequence_ID)
                    for Entrez_Comments in gene_record[0]['Entrezgene_comments']:
                        if 'Gene-commentary_comment' in Entrez_Comments:
                            for comments in Entrez_Comments['Gene-commentary_comment']:
                                if 'Gene-commentary_products' in comments:
                                    for product_per_assembly in comments['Gene-commentary_products']:                                
                                        if product_per_assembly['Gene-commentary_heading'] == 'mRNA Sequence':
                                            mRNA = product_per_assembly['Gene-commentary_accession']
                                            if mRNA == transcript_sequence_ID:  
                                                for protein_per_transcript_per_assemlby in product_per_assembly['Gene-commentary_products']:
                                                    protein_sequence_ID = protein_per_transcript_per_assemlby['Gene-commentary_accession']
                                                    print(protein_sequence_ID)

Organism:  Homo sapiens |Entrez ID:  127550 |Gene Name:  A3GALT2 |Gene Synonyms:  ['IGB3S', 'IGBS3S', 'A3GALT2P']
NM_001080438
NP_001073907
NM_001080438
NP_001073907
Organism:  Homo sapiens |Entrez ID:  553108 |Gene Name:  A3GALT2 |Gene Synonyms:  []


In [ ]:
for assembly_specific_info in gene_record[0]['Entrezgene_locus']:  
    for assembly_specific_transcript in assembly_specific_info['Gene-commentary_products']:
        transcript_sequence_ID = assembly_specific_transcript['Gene-commentary_accession']
        print(transcript_sequence_ID)